<h2>프로젝트 주제</h2>

<span style="font-size: 14pt; color: skyblue">seq2seq(Sequence-to-Sequence)을 이용한 한영 기계번역기 구현</span> <br />
<br />
RNN을 활용한 기본적인 seq2seq 예제를 구현해보고자 한다. <br/>
대부분의 seq2seq 흐름과 코드는 파이토치 공식 사이트에서 제공하는 NLP 튜토리얼(https://pytorch.org/tutorials/)에서 참고했다. <br/>
이후에 .ipynb 파일에 등장하는 코드 또한 공식 사이트에서 제공한 코드들을 베이스로 하고 있다.

<h2>주제 선정 이유 및 목표</h2>

주요 목표는 바로 작동하는 <span style="color: skyblue">한영 번역기</span>를 만드는 것이다 <br />
일단, 우리가 평소에 사용하는 수준의 번역기를 만드는 것은 어려울 것이라고 생각된다. 아무래도, 한국어는 대한민국의 독자적인 언어로서, <br /> 다른 영어권의 언어와는 어순도 다르며, 고려해야 할 요소가 셀 수 없이 많기 때문이다. <br />
그리하여, 일단 목표 자체는 작동하는 한영 번역기로 소박하게 설정하였다.<br />
그리고, 비교를 위해서 파이토치에서 제공하는 영어-프랑스어 번역기도 간단하게 구현할 예정이다. <br />
<br />
본래, pytorch로 동아리에서 AI 스터디에서 시간을 보내고는 했다. 그곳에서, Linear Regression부터 시작하여, CNN, Autoencoder, RNN, GAN 등을 이용하여, <br /> 여러가지 Task를 구현했었다. 이번에는 개인적으로 그보다 조금 더 복잡한 프로젝트를 시도해보고 싶었다. <br />
그중에서 눈길을 끌었던 것은 바로 seq2seq(Sequence-to-Sequence)였다. <br />

<h4>seq2seq에 대한 간략한 정보</h4> <br/>
Sequence-to-Sequence는 입력 시퀀스로부터 출력 시퀀스를 출력하며, 이를 각각 질문과 대답으로 설정한다면, Chatbot(챗봇)이나 기계번역 등서 다양하게 이용된다. 시퀸스-투-시퀸스(Sequence-to-Sequence)는 흔히 줄여서 seq2seq라고 쓴다. 이 앞에서 설명할 것은 대부분 RNN에서 언급한 내용이지만, 어떻게 이를 조립하느냐에 따라  seq2seq가 만들어진다. <br /> <br />

seq2seq는 RNN 셀로 이루어진 크게 '인코더'와 '디코더' 로 나누어진다. 인코더는 입력한 모든 단어를 순차적으로 압축해서 하나의 벡터로 만든다. 이 압축된 벡터를 Context vector라고 한다. 그리고 디코더는 Context vector를 받아서 번역된 단어를 한 개씩 순차적으로 출력한다. 실제로는 성능 때문에 RNN 셀이 아닌 LSTM 혹은 GRU 셀로 구성된다. 이번 프로젝트에서도 RNN이 아닌 GRU 셀을 사용할 예정이다. <br /> <br />

<h5>seq2seq 등장 배경</h5> <br />
English: I am not the black cat. <br />
French: Je ne suis pas chat noir.
위 문장에서 눈 여겨 봐야할 점은 2가지이다.
<br /><br />
1. black cat이 프랑스어로는 순서가 chat(=cat) noir(=black)으로 뒤바뀌었다. <br />
2. 영어에서 not을 프랑스어에서는 ne/pas로 2글자를 이용한다. <br /><br />
이러한 문제들 word by word로 직역한다고 했을 때, 번역기가 절대로 해결해 낼 수 없는 문제이다. 그리하여, 단순히 한 단어가 가지는 의미를 학습하여 직역하는 것 아니라, 한 문장이 전체적으로 내포하는 의미를 담아낸 이후에 이를 풀어내는 과정이 필요하다고 판단한 것이다.
그리하여, seq2seq는 문장의 의미를 함축하여 context vector로 표현하는 Encoder와 context vector의 의미를 다시 풀어내는 Decoder를 가진 구조를 갖게 되었다 <br /> <br />

<h5>seq2seq train(학습)</h5> <br />
입력 문장은 단어 토큰화에 의해 단어 단위로 쪼개지고, 각각의 토큰은 RNN 셀의 각 시점이 된다. 그리고, 인코더 RNN 셀의 마지막 시점의 hidden state(=Context vector)를 디코더 RNN 셀로 넘겨준다. 즉, Context vector는 디코더 RNN 셀의 첫번째 hidden state로 사용되는 것이다. <br />

디코더는 시작할 때 sos라는 심볼, 끝날 때는 문장의 끝을 나타내는 eos라는 심볼을 갖는다. 디코더가 sos를 받으면 처음에 올 높은 확률의 단어를 예측한다. 그리고, 첫번째로 올 것이라 예측한 단어(je)는 다음 디코더 RNN 셀에 들어간다. 그리고 다음에 두번째로 올 단어(suis)는 그 다음 디코더 RNN 셀에 들어가고, 이러한 행위가 eos가 다음 단어로 나올 때까지 반복된다. <br />

훈련 과정에서는 디코더가 인코더에게 '컨텍스트 벡터'및 'sos I am a student'를 받았을 때, 'je suis étudiant eos'가 나와야 한다는 정답을 알려주는 방식으로 훈련을 진행한다. 하지만 테스트 과정은 컨텍스트 벡터와 sos의 입력만으로 다음 시점의 단어를 예측해야 한다. 

디코더는 첫번째 hidden state로 Context vector와 입력값인 sos를 받는다. 이 값들을 토대로, 다음에 올 단어를 예측한다. 예측한 단어는 다시 t+1 RNN 셀의 입력값이 된다. <br />
    
RNN 셀로부터 출력된 값은 dense layer과 함수를 거쳐서, 하나의 단어를 골라 예측해야 한다. 이때, softmax 함수를 사용할 수 있다. softmax 함수는 RNN 셀의 출력 벡터를 받아 각 단어별 확률값을 반환한다. 그리고, 디코더는 출력할 단어를 결정하게 된다. <br />

<h2>과제 수행하면서 느낀점</h2>

<br />
수업시간에 배웠던 딥러닝은 Tensorflow를 이용한 Linear Regression이었다. 그리고, 학습을 통해 상관관계를 분석하여, 유의미한 결과를 도출해내는 것이었다. <br />
<br />
하지만, 본인이 진행한 프로젝트는 수업시간에 배운 딥러닝과는 분명 다른 점이 많은 것 같다. <br />
그래서, 아무래도 (1), (2), (3), (4)와 같은 형식으로 step별로 파일을 나누는 것이 쉽지 않았던 것 같다. <br />
<br />
조금 더 완벽한 한영 번역기를 만들고 싶었다. 하지만, Pytorch 문서와 함께 seq2seq 구현을 시도해보고자 하니, <br />
작동하는 한영 번역기를 만드는 것 자체가 도전이었던 것 같다. <br />
또한, 데이터셋의 중요성도 깨닫게 되었다. 전처리는 학습만큼 많은 노력을 들여야 함과 동시에,<br /> 
전처리 또한, 모델의 완성도에 많은 기여를 한다는 것을 뼈저리게 느꼈다.<br />
<br />
